# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [37]:
# Imports here

#######################################################
# Common Modules
#######################################################

# Time measurement
import time
# JSON library
import json
# Dictionary that keeps its named entries sorted as they are initially inserted
from collections import OrderedDict

#######################################################
# PyTorch
#######################################################
import torch
# Neural networks (classes to define and manipulate neural networks
from torch import nn
# Optimization algorithms to train and improve neural networks
from torch import optim
# Abstracts the complexity of data loading in parallel
from torch.utils.data import DataLoader  
# Datasets, model architectures, and common image transformations for computer vision.
from torchvision import (
    datasets,             # Standard datasets for training and testing models
    transforms,           # Common image transformations (resizing, cropping, rotation, etc.)
    models                # Pre-trained models and architectures (AlexNet, VGG, ResNet etc.)
)

## Global Settings

In [47]:
# Common
DEBUG = False
CUDA_ONLY = True # If Cuda-support is available, ignore CPU

# Training
EPOCHS = 5
LEARNING_RATE = 0.001
DROPOUT_RATE = 0.2

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [39]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [48]:
# DONE: Define your transforms for the training, validation, and testing sets
random_rotation = 30
image_size = 224
means_per_channel = [0.485, 0.456, 0.406]
stddevs_per_channel = [0.229, 0.224, 0.225]

data_transforms = {
    'train':    transforms.Compose([
                    transforms.RandomRotation(random_rotation),
                    transforms.RandomResizedCrop(image_size),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Normalize(means_per_channel, stddevs_per_channel),
                ]),
    'valid':    transforms.Compose([
                    transforms.Resize(image_size + 30),
                    transforms.CenterCrop(image_size),
                    transforms.ToTensor(),
                    transforms.Normalize(means_per_channel, stddevs_per_channel),
                ]),
    'test':     transforms.Compose([
                    transforms.Resize(image_size + 30),
                    transforms.CenterCrop(image_size),
                    transforms.ToTensor(),
                    transforms.Normalize(means_per_channel, stddevs_per_channel),
                ])
}

# DONE: Load the datasets with ImageFolder
image_datasets = {
    'train': datasets.ImageFolder(train_dir, transform=data_transforms['train']),
    'valid': datasets.ImageFolder(valid_dir, transform=data_transforms['valid']),
    'test':  datasets.ImageFolder(test_dir,  transform=data_transforms['test'])
} 

# DONE: Using the image datasets and the trainforms, define the dataloaders
data_loaders =  {
    'train': DataLoader(image_datasets['train'], batch_size=64, shuffle=True),
    'valid': DataLoader(image_datasets['valid'], batch_size=64),
    'test':  DataLoader(image_datasets['test'],  batch_size=64)
} 

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [49]:
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
    
print(json.dumps(cat_to_name, indent=4)) if DEBUG else None

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [50]:
# DONE: Build and train your network

# DONE 1: Create and adjust a pretrained network to the current classification
# DONE 2: Define a function to train the network
# DONE 3: Train the network for each device

In [51]:
# DONE 1: Create and adjust a pretrained network to the current classification

# Get an instance of a pretrained model. e.g. VGG16, ResNet50, DenseNet121, etc.)
model = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)
print(model) if DEBUG else None

# Determine layer-sizes of the neural network
inputs_size = model.classifier.in_features
output_size = len(cat_to_name) 
hl1_size = inputs_size // 2 # 512
hl2_size = inputs_size // 4 # 256

# Freeze parameters to prevent backpropagation
for param in model.parameters():
    param.requires_grad = False

# Customize the model to the current classification and
# add dropout to prevent overfitting
classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(inputs_size, hl1_size)),
    ('relu1', nn.ReLU()),
    ('dropout1', nn.Dropout(p = DROPOUT_RATE)),
    ('fc2', nn.Linear(hl1_size, hl2_size)),
    ('relu2', nn.ReLU()),
    ('dropout2', nn.Dropout(p = DROPOUT_RATE)),
    ('fc3', nn.Linear(hl2_size, output_size)),
    ('output', nn.LogSoftmax(dim = 1))
]))
model.classifier = classifier

In [52]:
# DONE 2: Define a function to train the network
def train_network(model, criterion, optimizer, train_loader, test_loader, device='cpu'):
    print(f'Start training for device {device}')

    # Initialize variables
    steps = 0
    running_train_loss = 0
    print_every = 10

    # Set model to training mode
    model.train()

    # Start measurement
    start_time = time.time()
    
    # Iterate over all epochs to train the network
    for epoch in range(1, EPOCHS+1):
        # Get inputs and labels of the current batch
        train_loader = data_loaders['train']
        for train_inputs, train_labels in train_loader:
            steps += 1

            # Move inputs and labels to device
            train_inputs, train_labels = train_inputs.to(device), train_labels.to(device)

            # Forward pass through the neural network and compute the loss of the model
            train_log_outputs = model.forward(train_inputs)
            train_loss = criterion(train_log_outputs, train_labels)

            # Clear old gradients, backpropagate and update model parameters with computed gradient
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()

            # Add current loss to the running loss            
            running_train_loss += train_loss.item()
            
            # Test the network after n batches and calulate train loss and test loss/accuracy
            if steps % print_every == 0:
                # Initialize test loss and accuracy
                running_test_loss = 0
                accuracy = 0

                # Set model to evaluation mode
                model.eval()

                # Determine train/test-loss and accuracy with test data withoud gradient
                with torch.no_grad():
                    # Perform the steps for test similar to training (see above)
                    test_loader = data_loaders['test']
                    for test_inputs, test_labels in test_loader:
                        test_inputs, test_labels = test_inputs.to(device), test_labels.to(device)
                        test_log_outputs = model.forward(test_inputs)
                        test_loss = criterion(test_log_outputs, test_labels)
                        running_test_loss += test_loss.item()

                        # Calculate accuracy of the network so far
                        test_output = torch.exp(test_log_outputs)
                        top_k, top_class = test_output.topk(1, dim=1)
                        equals = top_class == test_labels.view(*top_class.shape)
                        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

                # Output network metrics
                print(f'Epoch {epoch}/{EPOCHS}.. '
                      f'Train loss: {running_train_loss/print_every:.3f}.. '
                      f'Test loss: {running_test_loss/len(test_loader):.3f}.. '
                      f'Test accuracy: {accuracy/len(test_loader):.3f}.. '
                      f'Time elapsed: {int(time.time()-start_time)}s')
        
                # Reset model to training mode and reset running_train_loss for next batch
                model.train()
                running_train_loss = 0

    end_time = time.time()  
    elapsed_time = int(end_time - start_time)  

    print(f'End training for device {device}, duration={elapsed_time}s')


In [53]:
# DONE 3: Train the network for each device

# Add cpu to devices when cuda is not supported or cuda-only is configured when available 
devices = ['cpu'] if not torch.cuda.is_available() or not CUDA_ONLY else []
# Add all available cuda devices
devices += [torch.cuda.get_device_name(i) for i in range(torch.cuda.device_count())] if torch.cuda.is_available() else []  

# Train on all relevant devices
for device in ['cpu', 'cuda:0'] if torch.cuda.is_available() else ['cpu']:
    print(f"Training for device {device} starts")
    
    # Set loss function for the model to negative log likelihood loss
    criterion = nn.NLLLoss()
    # Set the optimizer for the model
    optimizer = optim.Adam(model.classifier.parameters(), lr=LEARNING_RATE)
    
    model.to(device)

    train_network(model, criterion, optimizer, data_loaders['train'], data_loaders['test'], device)
    print(f"Training for device {device} finished")


Training for device cpu starts
Start training for device cpu
Epoch 1/5.. Train loss: 4.563.. Test loss: 4.478.. Test accuracy: 0.047.. Time elapsed: 62s
Epoch 1/5.. Train loss: 4.514.. Test loss: 4.380.. Test accuracy: 0.069.. Time elapsed: 128s
Epoch 1/5.. Train loss: 4.325.. Test loss: 4.159.. Test accuracy: 0.107.. Time elapsed: 194s
Epoch 1/5.. Train loss: 4.130.. Test loss: 3.918.. Test accuracy: 0.152.. Time elapsed: 257s
Epoch 1/5.. Train loss: 3.877.. Test loss: 3.606.. Test accuracy: 0.191.. Time elapsed: 322s
Epoch 1/5.. Train loss: 3.710.. Test loss: 3.281.. Test accuracy: 0.276.. Time elapsed: 387s
Epoch 1/5.. Train loss: 3.356.. Test loss: 2.978.. Test accuracy: 0.292.. Time elapsed: 453s
Epoch 1/5.. Train loss: 3.176.. Test loss: 2.679.. Test accuracy: 0.361.. Time elapsed: 519s
Epoch 1/5.. Train loss: 2.847.. Test loss: 2.326.. Test accuracy: 0.413.. Time elapsed: 584s
Epoch 1/5.. Train loss: 2.600.. Test loss: 2.046.. Test accuracy: 0.496.. Time elapsed: 649s
Epoch 2/5.

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [58]:
# TODO: Do validation on the test set

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [58]:
# TODO: Save the checkpoint


## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes